In [755]:
from knnmv_master.impute import KNNMVImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [853]:
train = pd.read_csv('Train/Train.csv')
health_camps = pd.read_csv('Train/Health_Camp_Detail.csv')
patient_profile = pd.read_csv('Train/Patient_Profile.csv')
first = pd.read_csv('Train/First_Health_Camp_Attended.csv')
second = pd.read_csv('Train/Second_Health_Camp_Attended.csv')
third = pd.read_csv('Train/Third_Health_Camp_Attended.csv')

test = pd.read_csv('test_l0Auv8Q.csv')

##### Patient Profile Data

In [854]:
patient_profile = patient_profile.replace('None', np.nan)

patient_profile['First_Interaction'] = pd.to_datetime(patient_profile['First_Interaction'])

In [855]:
# handling NaN columns, Income, Education_Score, Age, City_Type and Employer_Category
categorical_cols = ['City_Type','Employer_Category']

for i in categorical_cols:
    patient_profile[i].fillna('NaN', inplace=True)
    
# for i in categorical_cols:
#     temp_one_hot = pd.get_dummies(patient_profile[[i]], columns=[i])
#     patient_profile = patient_profile.join(temp_one_hot)
    
#     patient_profile = patient_profile.drop(columns=[i])

In [856]:
# numerical columns containing NaNs (to be impuuted)
missing_numerical_cols = ['Income', 'Education_Score', 'Age']

all_numerical_cols = ['Online_Follower', 'LinkedIn_Shared', 'Twitter_Shared',
       'Facebook_Shared', 'Income', 'Education_Score', 'Age']

# all_numerical_cols = ['Online_Follower', 'LinkedIn_Shared', 'Twitter_Shared',
#        'Facebook_Shared', 'Income', 'Education_Score', 'Age',
#        'City_Type_A', 'City_Type_B', 'City_Type_C',
#        'City_Type_D', 'City_Type_E', 'City_Type_F', 'City_Type_G',
#        'City_Type_H', 'City_Type_I', 'City_Type_NaN', 'Employer_Category_BFSI',
#        'Employer_Category_Broadcasting', 'Employer_Category_Consulting',
#        'Employer_Category_Education', 'Employer_Category_Food',
#        'Employer_Category_Health', 'Employer_Category_Manufacturing',
#        'Employer_Category_NaN', 'Employer_Category_Others',
#        'Employer_Category_Real Estate', 'Employer_Category_Retail',
#        'Employer_Category_Software Industry', 'Employer_Category_Technology',
#        'Employer_Category_Telecom', 'Employer_Category_Transport']

for i in missing_numerical_cols:
    patient_profile[i] = patient_profile[i].astype(float)
#     patient_profile[i] = patient_profile.fillna(patient_profile[i].median())


In [858]:
# # technique 1
# temp = patient_profile[all_numerical_cols].iloc[0:2000]
# # documentation: https://github.com/massibelloni-medium/knnmv
# knnmv_imp = KNNMVImputer(strategy="median", k=5, l=0.25)
# temp = knnmv_imp.fit_transform(temp.to_numpy(), verbose=True)

# technique 2
from xgboost import XGBRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer = IterativeImputer(random_state=0, max_iter=100, imputation_order='descending')
temp = imputer.fit_transform(patient_profile[all_numerical_cols])

# common for both 1 and 2
temp = pd.DataFrame(temp, columns=all_numerical_cols)

patient_profile = pd.concat([patient_profile[set(patient_profile.columns).difference(set(all_numerical_cols))],
                                                                       temp], axis=1)

In [859]:
patient_profile.isna().sum()
patient_profile.dtypes

Employer_Category    0
City_Type            0
First_Interaction    0
Patient_ID           0
Online_Follower      0
LinkedIn_Shared      0
Twitter_Shared       0
Facebook_Shared      0
Income               0
Education_Score      0
Age                  0
dtype: int64

Employer_Category            object
City_Type                    object
First_Interaction    datetime64[ns]
Patient_ID                    int64
Online_Follower             float64
LinkedIn_Shared             float64
Twitter_Shared              float64
Facebook_Shared             float64
Income                      float64
Education_Score             float64
Age                         float64
dtype: object

#### Health Camp Data

In [860]:
health_camps['Camp_Start_Date'] = pd.to_datetime(health_camps['Camp_Start_Date'])
health_camps['Camp_End_Date'] = pd.to_datetime(health_camps['Camp_End_Date'])

#### Train Data

In [861]:
train['Registration_Date'] = pd.to_datetime(train['Registration_Date'])

test['Registration_Date'] = pd.to_datetime(test['Registration_Date'])

In [862]:
# dropping data points where Registration_Date is NaN
train.shape
train = train.dropna().reset_index(drop=True)
train.shape

(75278, 8)

(74944, 8)

### Merging the datasets

In [863]:
train.shape
train = train.merge(patient_profile, on=['Patient_ID'], how='left')
train.shape

train.shape
train = train.merge(health_camps, on=['Health_Camp_ID'], how='left')
train.shape

(74944, 8)

(74944, 18)

(74944, 18)

(74944, 23)

In [864]:
test.shape
test = test.merge(patient_profile, on=['Patient_ID'], how='left')
test.shape

test.shape
test = test.merge(health_camps, on=['Health_Camp_ID'], how='left')
test.shape

(35249, 8)

(35249, 18)

(35249, 18)

(35249, 23)

In [865]:
first['First_Favorable'] = 1
second['Second_Favorable'] = 1
third['Third_Favorable'] = 1

train.shape

train = train.merge(first[['Patient_ID','Health_Camp_ID','First_Favorable']], 
                              on=['Patient_ID','Health_Camp_ID'], how='left')

train = train.merge(second[['Patient_ID','Health_Camp_ID','Second_Favorable']], 
                              on=['Patient_ID','Health_Camp_ID'], how='left')

train = train.merge(third[['Patient_ID','Health_Camp_ID','Third_Favorable']], 
                              on=['Patient_ID','Health_Camp_ID'], how='left')

train.shape

(74944, 23)

(74944, 26)

In [866]:
# # Validating data correctness
# for i in ['First','Second','Third']:    
#     train[~train[i+'_Favorable'].isnull()]['Category1'].unique()

# for i in ['First','Second','Third']:    
#     train[train['Category1'] == i]['First_Favorable'].unique()
#     train[train['Category1'] == i]['Second_Favorable'].unique()
#     train[train['Category1'] == i]['Third_Favorable'].unique()

In [867]:
train['Interaction_to_Registration'] = (train['Registration_Date'] - train['First_Interaction']).dt.days
train['Registration_to_Camp_Start_Date'] = (train['Camp_Start_Date'] - train['Registration_Date']).dt.days
train['Registration_to_Camp_End_Date'] = (train['Camp_End_Date'] - train['Registration_Date']).dt.days
train['Interaction_to_Camp_Start_Date'] = (train['Camp_Start_Date'] - train['First_Interaction']).dt.days
train['Interaction_to_Camp_End_Date'] = (train['Camp_End_Date'] - train['First_Interaction']).dt.days


test['Interaction_to_Registration'] = (test['Registration_Date'] - test['First_Interaction']).dt.days
test['Registration_to_Camp_Start_Date'] = (test['Camp_Start_Date'] - test['Registration_Date']).dt.days
test['Registration_to_Camp_End_Date'] = (test['Camp_End_Date'] - test['Registration_Date']).dt.days
test['Interaction_to_Camp_Start_Date'] = (test['Camp_Start_Date'] - test['First_Interaction']).dt.days
test['Interaction_to_Camp_End_Date'] = (test['Camp_End_Date'] - test['First_Interaction']).dt.days

Combining First and Second as one classification problem and the third as another

In [868]:
# first or second favorable
train.loc[(train['First_Favorable'] == 1) | (train['Second_Favorable'] == 1), 'Target_1'] = 1
train['Target_1'].fillna(0, inplace=True)

# third favorable
train['Target_2'] = train['Third_Favorable'].fillna(0)

# # Solving each case separately (training 3 different models)
# train['Target_1'] = train['First_Favorable'].fillna(0)
# train['Target_2'] = train['Second_Favorable'].fillna(0)
# train['Target_3'] = train['Third_Favorable'].fillna(0)

In [869]:
drop_cols = ['Registration_Date','First_Interaction','Camp_Start_Date','Camp_End_Date',
             'First_Favorable','Second_Favorable','Third_Favorable']

train = train.drop(columns=drop_cols)

test = test.drop(columns=['Registration_Date','First_Interaction','Camp_Start_Date','Camp_End_Date'])

In [870]:
# Rounding off age, income and education_score columns
for i in missing_numerical_cols:
    train[i] = train[i].round(0)
    test[i] = test[i].round(0)

In [871]:
train.to_csv('Interm/train.csv', index=0)
test.to_csv('Interm/test.csv', index=0)